# Day 11

## part 1

- stones in a line w/ numbers
- they change with these rules
    - 0 -> 1
    - even # digits -> splits into 2 stones:
        - Left half on left stone
        - Right half on right stone
        - Leading zeros are removed
    - otherwise replace with new stone which has 2024 times the value
- ordering is preserved
- how many stones are there after 25 blinks?


In [24]:
import logging

from tqdm import tqdm

from advent_of_code_utils.advent_of_code_utils import (
    parse_from_file, ParseConfig as PC, markdown
)

log = logging.getLogger('day 11')
logging.basicConfig(level=logging.INFO)

In [4]:
# let's test the example first
stones = [125, 17]

# I have a feeling I'm going to need to be smarter but let's just try and
# brute force it first
def blink(stones: list[int]) -> list[int]:
    """returns the stones after blinking"""
    temp = []
    for stone in stones:
        # 0 case
        if stone == 0:
            temp.append(1)
            continue
        # even digit case
        str_stone = str(stone)
        if len(str_stone) % 2 == 0:
            half = len(str_stone) // 2
            left = int(str_stone[:half])
            right = int(str_stone[half:])
            temp.extend([left, right])
        # otherwise
        else:
            temp.append(stone * 2024)
    log.debug(f'new length: {len(temp)}')
    return temp

log.setLevel(logging.DEBUG)
for _ in range(25):
    stones = blink(stones)

DEBUG:day 11:new length: 3
DEBUG:day 11:new length: 4
DEBUG:day 11:new length: 5
DEBUG:day 11:new length: 9
DEBUG:day 11:new length: 13
DEBUG:day 11:new length: 22
DEBUG:day 11:new length: 31
DEBUG:day 11:new length: 42
DEBUG:day 11:new length: 68
DEBUG:day 11:new length: 109
DEBUG:day 11:new length: 170
DEBUG:day 11:new length: 235
DEBUG:day 11:new length: 342
DEBUG:day 11:new length: 557
DEBUG:day 11:new length: 853
DEBUG:day 11:new length: 1298
DEBUG:day 11:new length: 1951
DEBUG:day 11:new length: 2869
DEBUG:day 11:new length: 4490
DEBUG:day 11:new length: 6837
DEBUG:day 11:new length: 10362
DEBUG:day 11:new length: 15754
DEBUG:day 11:new length: 23435
DEBUG:day 11:new length: 36359
DEBUG:day 11:new length: 55312


In [10]:
# cool let's see if brute forcing works for the real deal
log.setLevel(logging.INFO)
parser = PC(' ', int)
stones = parse_from_file('day_11.txt', parser)
for _ in tqdm(range(25)):
    stones = blink(stones)

INFO:advent_of_code_utils.py:8 items loaded from "day_11.txt"
100%|██████████| 25/25 [00:00<00:00, 71.80it/s] 


In [11]:
markdown(f'after 25 blinks there are: {len(stones)} stones!')

after 25 blinks there are: 197357 stones!

## part 2

- here we go: 75 blinks let's gooooo

In [12]:
stones = parse_from_file('day_11.txt', parser)
for _ in tqdm(range(75)):
    stones = blink(stones)

INFO:advent_of_code_utils.py:8 items loaded from "day_11.txt"
 52%|█████▏    | 39/75 [01:49<01:41,  2.81s/it] 


KeyboardInterrupt: 

- as I suspected! I may actually have to put in some thought *:P
- first thing to consider is the creation of 0s - stones don't interact with eachother so it's easy to determine what the future blinks will do to each one. And 0s are going to come up alot.
- wait let's check that.

In [13]:

log.info(f'{stones.count(0)=}')

INFO:day 11:stones.count(0)=3512537


yup that's a lot of completely predictable stones that are wasting our time

In [21]:
# let's see what happens to a 0
stones = [0]
for count in range(1, 5):
    stones = blink(stones)
    log.info(f'{count=} {stones=}')

INFO:day 11:count=1 stones=[1]
INFO:day 11:count=2 stones=[2024]
INFO:day 11:count=3 stones=[20, 24]
INFO:day 11:count=4 stones=[2, 0, 2, 4]


so each 0 becomes 4 stones after 4 iterations and produces:
- 2x 2
- 1x 4
- and another 0

Let's just count the instances of each value rather than store each individual value to help reduce the amount of info we need to store

In [ ]:
memo = {}  # gonna try and memoise as well for extra speeeed

def smart_blink(totals: dict[int: int]) -> dict[int: int]:
    """
    enacts blink but only keeps track of totals since ordering doesn't matter
    """
    temp = {}
    for value, quantity in totals.items():
        # figure out what to add
        if value in memo:
            additions = memo[value]
        else:
            additions = blink([value])
            memo.update({value: additions})
        
        # update totals
        for value in additions:
            if value not in temp:
                temp.update({value: quantity})
            else:
                temp[value] += quantity
    log.debug(f'new total: {sum(temp.values())}')
    return temp

# let's check it works by redoing the example for 25 runs
log.setLevel(logging.INFO)
stones = [125, 17]
totals = {value: 1 for value in stones}
log.debug(f'{totals=}, {stones=}')
for _ in range(25):
    totals = smart_blink(totals)
    stones = blink(stones)  # just for debug logs to compare
    log.debug(f'{totals=}, {stones=}')

log.info(f'memoised: {len(memo)} stones')
log.info(f'after 25 blinks that created {sum(totals.values())}')

INFO:day 11:memoised: 76 stones
INFO:day 11:after 25 blinks that created 55312


In [32]:
# cool that seems to work let's solve!
log.setLevel(logging.INFO)
stones = parse_from_file('day_11.txt', parser)
totals = {value: 1 for value in stones}
for _ in tqdm(range(75)):
    totals = smart_blink(totals)
log.info(f'memoised: {len(memo)} stones')
markdown(f'after 75 blinks that created {sum(totals.values())}')

INFO:advent_of_code_utils.py:8 items loaded from "day_11.txt"
100%|██████████| 75/75 [00:00<00:00, 380.79it/s]
INFO:day 11:memoised: 3908 stones


after 75 blinks that created 234568186890978